## Cargue de librerias

In [1]:
!pip install translate


[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
import pandas as pd
from translate import Translator
import json

In [5]:
df = pd.read_csv("../preprocessing/datos_procesados.csv")

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class ModeloSimilitudTextos:
    def __init__(self):
        self.vectorizer = TfidfVectorizer()
        self.X_train = None
        self.y_train = None

    def entrenar_modelo(self, corpus, etiquetas):
        self.X_train = self.vectorizer.fit_transform(corpus)
        self.y_train = etiquetas

    def encontrar_similitud(self, texto):
        vector_texto = self.vectorizer.transform([texto])
        similitudes = cosine_similarity(vector_texto, self.X_train)
        indice_max_similitud = similitudes.argmax()
        resultado_aproximado = self.y_train[indice_max_similitud]
        return resultado_aproximado

In [7]:
modelo = ModeloSimilitudTextos()
modelo.entrenar_modelo(df['limpieza'], df['Titulo'])

In [29]:
def entontar_titulo(texto):
    translator = Translator(from_lang="es" ,to_lang="en")
    translated_text = translator.translate(texto)
    resultado_aproximado = modelo.encontrar_similitud(translated_text)
    resultado_aproximado={"respuesta": resultado_aproximado}
    json_data = json.dumps(resultado_aproximado)
    return json_data

In [30]:
entontar_titulo("Gamoray Peter Quill tienen que defender el universo")

'{"respuesta": "Guardians of the Galaxy Vol. 3"}'

# Disponibilizar

In [ ]:
from flask import Flask, request, jsonify
import joblib

app = Flask(__name__)

# Cargar el modelo desde disco

# Definir el endpoint para las inferencias
@app.route('/predict', methods=['POST'])
def predict():
    data = request.json  
    predictions = entontar_titulo(data)
    #return jsonify(predictions.tolist())
    return predictions
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jul/2023 23:20:14] "POST /predict HTTP/1.1" 200 -
